Cross-functional team, 1 DS

MLFlow setup:
 - Tracking server: local
 - Backend store: sqlite
 - artifact store: local fs

The experiments could be tracked via MLFlow UI.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:
`mlflow server --backend-store-uri sqlite:///backend.db`

In [1]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [3]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")
mlflow.search_experiments()

tracking URI: 'http://127.0.0.1:5000'


[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1716742854267, experiment_id='0', last_update_time=1716742854267, lifecycle_stage='active', name='Default', tags={}>]

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

default artifacts URI: 'mlflow-artifacts:/1/81e87f6e731e4ef3a6f09419fa28ffa7/artifacts'


/home/codespace/anaconda3/envs/exp-trackin-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:11: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/codespace/anaconda3/envs/exp-trackin-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [5]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1716743021789, experiment_id='1', last_update_time=1716743021789, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1716742854267, experiment_id='0', last_update_time=1716742854267, lifecycle_stage='active', name='Default', tags={}>]

In [7]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")


In [9]:
client.search_registered_models()

[]

In [21]:
run_id = client.search_runs(experiment_ids=1)[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2024/05/26 17:11:45 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1716743505255, current_stage='None', description='', last_updated_timestamp=1716743505255, name='iris-classifier', run_id='81e87f6e731e4ef3a6f09419fa28ffa7', run_link='', source='mlflow-artifacts:/1/81e87f6e731e4ef3a6f09419fa28ffa7/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>

In [22]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1716743505150, description='', last_updated_timestamp=1716743505255, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1716743505255, current_stage='None', description='', last_updated_timestamp=1716743505255, name='iris-classifier', run_id='81e87f6e731e4ef3a6f09419fa28ffa7', run_link='', source='mlflow-artifacts:/1/81e87f6e731e4ef3a6f09419fa28ffa7/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>], name='iris-classifier', tags={}>]